In [1]:
from utils import get_test_table
import numpy as np

In [2]:
X_train, X_test, y_train, y_test = get_test_table()

Encoding the date column

In [3]:
import pandas as pd

# Assuming 'X_train' and 'X_test' are pandas DataFrames and have a date column named 'date'
X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

# Sort data by date
X_train = X_train.sort_values(by='date')
X_test = X_test.sort_values(by='date')

# Assign a unique number to each date
unique_dates = pd.concat([X_train['date'], X_test['date']]).unique()
date_to_number = {date: i for i, date in enumerate(sorted(unique_dates))}
X_train['date'] = X_train['date'].map(date_to_number).astype('float32')
X_test['date'] = X_test['date'].map(date_to_number).astype('float32')

# Convert the rest of your dataframe to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Reshape for LSTM input
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

Model

In [4]:
""" from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import backend as K

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmse])

model.fit(X_train, y_train, epochs=5, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False) """

" from keras.models import Sequential\nfrom keras.layers import LSTM, Dense\nfrom keras import backend as K\n\ndef rmse(y_true, y_pred):\n    return K.sqrt(K.mean(K.square(y_pred - y_true)))\n\nmodel = Sequential()\nmodel.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))\nmodel.add(Dense(1))\nmodel.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmse])\n\nmodel.fit(X_train, y_train, epochs=5, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False) "

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import backend as K

param_grid = {
    'batch_size': [8, 16],
    'epochs': [3, 5]
}

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def create_compile_model(optimizer):
    model = Sequential()
    model.add(LSTM(80, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[rmse])
    return model

best_score = None
best_params = None

for batch_size in param_grid['batch_size']:
    for epochs in param_grid['epochs']:

            model = create_compile_model('adam')

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1, shuffle=False)

            score = model.evaluate(X_test, y_test, verbose=0)

            if best_score is None or score < best_score:
                best_score = score
                best_params = {'batch_size': batch_size, 'epochs': epochs}

print(f'Best score: {best_score}, with parameters: {best_params}')


Epoch 1/3
1247/1247 [==============================] - 26s 13ms/step - loss: 64.8629 - rmse: 7.0531 - val_loss: 59.6048 - val_rmse: 6.7604
Epoch 2/3
1247/1247 [==============================] - 12s 10ms/step - loss: 63.3462 - rmse: 7.0310 - val_loss: 59.4916 - val_rmse: 6.7606
Epoch 3/3
1247/1247 [==============================] - 15s 12ms/step - loss: 63.2302 - rmse: 7.0219 - val_loss: 59.4611 - val_rmse: 6.7643
Epoch 1/5
1247/1247 [==============================] - 29s 13ms/step - loss: 64.6607 - rmse: 7.0520 - val_loss: 59.1240 - val_rmse: 6.7955
Epoch 2/5
1247/1247 [==============================] - 14s 11ms/step - loss: 63.3535 - rmse: 7.0282 - val_loss: 59.1210 - val_rmse: 6.7913
Epoch 3/5
1247/1247 [==============================] - 14s 11ms/step - loss: 63.2151 - rmse: 7.0231 - val_loss: 59.1210 - val_rmse: 6.7959
Epoch 4/5
1247/1247 [==============================] - 15s 12ms/step - loss: 63.1592 - rmse: 7.0196 - val_loss: 59.1204 - val_rmse: 6.7924
Epoch 5/5
1247/1247 [======